# Importing Libraries

In [2]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# Data Collection:
* **Collect data from api**
* **Save data from tsv to csv**

In [4]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"]).dropna()
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"]).dropna()

--2023-07-29 07:26:16--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2023-07-29 07:26:16 (9.35 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2023-07-29 07:26:18--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[======

# Data pre-processing:
* **Converting "type" column to numerical**
* **Creating Tensorflow dataset**
* **Preparing parameter and variable for training**

In [7]:
train_df["type"] = pd.factorize(train_df["type"])[0]
test_df["type"] = pd.factorize(test_df["type"])[0]

In [8]:
train_labels =  train_df["type"].values
train_ds = tf.data.Dataset.from_tensor_slices(
    (train_df["msg"].values, train_labels)
)
test_labels =  test_df["type"].values
test_ds = tf.data.Dataset.from_tensor_slices(
    (test_df["msg"].values, test_labels)
)
train_ds.element_spec

(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

In [9]:
BUFFER_SIZE = 100
BATCH_SIZE = 32
train_ds = train_ds.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE) 
test_ds = test_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Training, Validation, and Testing:
* **Creating model layer**
* **Predict 1 message as Testing**
* **Predict multiple messages as Testing**

In [10]:
vec = tf.keras.layers.TextVectorization(
    output_mode='int',
    max_tokens=1000,
    output_sequence_length=1000,
)

vec.adapt(train_ds.map(lambda text, label: text))

In [11]:
model = tf.keras.Sequential([
    vec,
    tf.keras.layers.Embedding(
        len(vec.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)

In [12]:
history = model.fit(
    train_ds,
    validation_data=test_ds,
    validation_steps=30,
    epochs=10,
)

Epoch 1/10
131/131 [==============================] - 238s 2s/step - loss: 0.6137 - accuracy: 0.8660 - val_loss: 0.4952 - val_accuracy: 0.8604
Epoch 2/10
131/131 [==============================] - 214s 2s/step - loss: 0.3782 - accuracy: 0.8660 - val_loss: 0.2166 - val_accuracy: 0.8604
Epoch 3/10
131/131 [==============================] - 216s 2s/step - loss: 0.1503 - accuracy: 0.9423 - val_loss: 0.1018 - val_accuracy: 0.9698
Epoch 4/10
131/131 [==============================] - 216s 2s/step - loss: 0.0870 - accuracy: 0.9799 - val_loss: 0.0707 - val_accuracy: 0.9812
Epoch 5/10
131/131 [==============================] - 214s 2s/step - loss: 0.0638 - accuracy: 0.9856 - val_loss: 0.0646 - val_accuracy: 0.9844
Epoch 6/10
131/131 [==============================] - 214s 2s/step - loss: 0.0512 - accuracy: 0.9878 - val_loss: 0.0651 - val_accuracy: 0.9865
Epoch 7/10
131/131 [==============================] - 214s 2s/step - loss: 0.0425 - accuracy: 0.9899 - val_loss: 0.0540 - val_accuracy: 0.9896

In [14]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    ps = model.predict([pred_text])
    print(ps)
    p = ps[0][0]
    return [p, "ham" if p <0.5 else "spam"]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 195ms/step
[[-6.810312]]
[-6.810312, 'ham']


In [15]:
def test_predictions():
    test_messages = ["how are you doing today",
                     "sale today! to stop texts call 98912460324",
                     "i dont want to go. can we try it a different day? available sat",
                     "our new mobile video service is live. just install on your phone to start watching.",
                     "you have won £1000 cash! call to claim your prize.",
                     "i'll bring it tomorrow. don't forget the milk.",
                     "wow, is your arm alright. that happened to me one time too"
                    ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    correct_predictions = 0
    total_predictions = len(test_messages)

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] == ans:
            correct_predictions += 1

    accuracy_percentage = (correct_predictions / total_predictions) * 100
    print("Correct Prediction Percentage: {:.2f}%".format(accuracy_percentage))

test_predictions()

1/1 [==============================] - 0s 198ms/step
[[-6.810312]]
1/1 [==============================] - 0s 200ms/step
[[0.33513367]]
1/1 [==============================] - 0s 227ms/step
[[-10.418703]]
1/1 [==============================] - 0s 195ms/step
[[5.286251]]
1/1 [==============================] - 0s 228ms/step
[[6.0942535]]
1/1 [==============================] - 0s 199ms/step
[[-9.227549]]
1/1 [==============================] - 0s 195ms/step
[[-9.791139]]
Correct Prediction Percentage: 85.71%
